In [78]:
import os
import pandas as pd
import numpy as np

In [79]:
df = pd.read_csv('butler.csv')
original = df.copy()
df.drop(list(df.filter(regex = 'RANK')), axis = 1, inplace = True)


In [80]:
df['next_pts'] = df['PTS'].shift(-1)

In [81]:
df.dtypes[df.dtypes == "object"]
df.drop(["WL", "SEASON_YEAR"], axis=1)

,TEAM_ABBREVIATION,GAME_DATE,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,OPPONENT,HOME,next_pts
0,PHI,2022-10-18T00:00:00,37.271667,9,14,0.643,5,9,0.556,12,...,0.171,0.000,1.000,0.200,0.800,0.111,0.889,BOS,0,31.0
1,PHI,2022-10-20T00:00:00,40.331667,13,24,0.542,1,7,0.143,4,...,0.387,0.250,0.750,1.000,0.000,0.308,0.692,MIL,1,12.0
2,PHI,2022-10-22T00:00:00,40.298333,4,18,0.222,1,6,0.167,3,...,0.500,0.000,1.000,1.000,0.000,0.250,0.750,SAS,1,29.0
3,PHI,2022-10-24T00:00:00,34.650000,10,18,0.556,5,10,0.500,4,...,0.138,0.200,0.800,0.200,0.800,0.200,0.800,IND,1,18.0
4,PHI,2022-10-26T00:00:00,35.650000,5,12,0.417,3,6,0.500,5,...,0.222,0.000,1.000,0.000,1.000,0.000,1.000,TOR,0,11.0
5,PHI,2022-10-28T00:00:00,33.716667,4,9,0.444,0,3,0.000,3,...,0.727,0.000,1.000,0.000,0.000,0.000,1.000,TOR,0,15.0
6,PHI,2022-10-29T00:00:00,35.993333,2,13,0.154,0,5,0.000,11,...,0.000,0.000,1.000,0.000,0.000,0.000,1.000,CHI,0,23.0
7,PHI,2022-10-31T00:00:00,37.616667,8,17,0.471,2,6,0.333,5,...,0.435,0.000,1.000,0.000,1.000,0.000,1.000,WAS,0,24.0
8,PHI,2022-11-02T00:00:00,35.066667,8,18,0.444,3,8,0.375,5,...,0.333,0.000,1.000,0.667,0.333,0.250,0.750,WAS,1,21.0
9,PHI,2022-12-05T00:00:00,38.410000,4,19,0.211,4,11,0.364,9,...,0.000,0.000,0.000,0.000,1.000,0.000,1.000,HOU,0,28.0


In [82]:
df['team_code'] = df['TEAM_ABBREVIATION'].astype("category").cat.codes
df['opponent_code'] = df['OPPONENT'].astype("category").cat.codes
fulldf = df.copy()
df = df.dropna()

In [83]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler


In [84]:
rr = Ridge(alpha=0.8)
svrmod = SVR(kernel="linear")
split = TimeSeriesSplit(n_splits=5)
mor = MultiOutputRegressor(svrmod)

sfs = SequentialFeatureSelector(rr, n_features_to_select=15, direction="forward", cv=split, n_jobs=4)

In [85]:
removed_cols = ["next_pts", "SEASON_YEAR", "TEAM_ABBREVIATION", "OPPONENT", "GAME_DATE", "WL", "team_code", 'FTM']
selected_cols = df.columns[~df.columns.isin(removed_cols)]
df.to_csv('kcsv.csv', index=False)

In [86]:

scaler = MinMaxScaler()
df.loc[:, selected_cols] = scaler.fit_transform(df[selected_cols])

In [87]:
df.describe()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,HOME,next_pts,team_code,opponent_code
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,...,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.0,57.000000
mean,0.540252,0.451754,0.428070,0.422871,0.464912,0.528509,0.514546,5.456140,0.481659,0.733228,...,0.174281,0.866211,0.374561,0.520175,0.297024,0.702976,0.561404,20.719298,0.0,0.498246
std,0.184074,0.200413,0.192497,0.192741,0.264906,0.248901,0.254934,2.652964,0.255959,0.266509,...,0.244711,0.200085,0.347940,0.369681,0.238833,0.238833,0.500626,6.723505,0.0,0.319738
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,0.000000
25%,0.455788,0.333333,0.300000,0.309524,0.333333,0.375000,0.350140,3.000000,0.272727,0.500000,...,0.000000,0.750000,0.000000,0.250000,0.133253,0.600240,0.000000,16.000000,0.0,0.200000
50%,0.553520,0.416667,0.400000,0.421569,0.500000,0.500000,0.525210,5.000000,0.454545,0.750000,...,0.000000,1.000000,0.333000,0.500000,0.300120,0.699880,1.000000,20.000000,0.0,0.480000
75%,0.641944,0.583333,0.550000,0.553221,0.666667,0.750000,0.700280,7.000000,0.636364,1.000000,...,0.374813,1.000000,0.500000,0.800000,0.399760,0.866747,1.000000,26.000000,0.0,0.760000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,38.000000,0.0,1.000000


In [88]:
sfs3 = SequentialFeatureSelector(svrmod, n_features_to_select=10, direction="forward", cv=split, n_jobs=4)

In [89]:
sfs3.fit(df[selected_cols], df["next_pts"])


SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                          estimator=SVR(kernel='linear'),
                          n_features_to_select=10, n_jobs=4)

In [90]:
predictors = list(selected_cols[sfs3.get_support()])
predictors

['FTA',
 'AST',
 'PCT_OREB',
 'PTS_PAINT',
 'OREB_PCT',
 'PCT_PTS_2PT',
 'PCT_PTS_OFF_TOV',
 'PCT_PTS_PAINT',
 'PCT_AST_2PM',
 'PCT_UAST_2PM']

In [138]:
def backtest(data, model, predictors, start=4, step=1):
    allpreds = []
    games = sorted(data["GAME_DATE"].unique())

    for i in range(start, len(games), step):
        current_game = games[i]
        train = data[data["GAME_DATE"] < current_game]
        test = data[data["GAME_DATE"] == current_game]

        model.fit(train[predictors], train[['next_pts', 'FTM']])
        preds = model.predict(test[predictors])
        combined = pd.concat([test[["next_pts", 'FTM']], pd.Series(preds[0][0], index=test.index), pd.Series(preds[0][1], index=test.index)], axis=1)
        # combined.columns = ["actual", "prediction"]
        allpreds.append(combined)

    return pd.concat(allpreds)

In [139]:
predictions = backtest(df, mor, predictors)



In [140]:
predictions

,next_pts,FTM,0,1
4,11.0,5,23.357699,4.288614
5,15.0,3,18.607305,3.722890
6,23.0,11,17.860866,5.089905
7,24.0,5,17.551009,4.360379
8,21.0,5,19.918223,4.780761
9,28.0,9,21.978008,5.413339
10,19.0,7,21.847354,5.491129
11,21.0,4,21.214697,5.354551
12,27.0,3,21.354282,5.496045
13,14.0,7,21.940437,5.419682


In [ ]:
svrmod.predict(df.iloc[[43]][predictors])


array([21.02200388])

In [ ]:
original.iloc[[44]]["PTS"]

44    23
Name: PTS, dtype: int64

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

43.403120260669894

In [ ]:
df["next_pts"].describe()

count    57.000000
mean     20.719298
std       6.723505
min       5.000000
25%      16.000000
50%      20.000000
75%      26.000000
max      38.000000
Name: next_pts, dtype: float64

In [ ]:
def player_history(df):
    df = df.sort_values("GAME_DATE")
    df["game_number"] = range(0, df.shape[0])
    df['pts_corr'] = list(df[['game_number', 'PTS']].expanding().corr().loc[(slice(None), "game_number"), "PTS"])
    df['pts_corr'].fillna(0, inplace=True)

    df['pts_diff'] = df['PTS'] / df['PTS'].shift(1)
    df['pts_diff'].fillna(df['pts_diff'].mean(), inplace=True)

    df['pts_diff'][df['pts_diff'] == np.inf] = 1

    return df


In [ ]:
df = df.groupby("SEASON_YEAR", group_keys=False).apply(player_history)

In [ ]:
# pd.Series(svrmod.coef_, index=predictors).sort_values()

In [ ]:
diff = predictions['actual'] - predictions['prediction']
diff

4    -13.040584
5     -2.962843
6      5.182297
7      5.874556
8      1.226769
9      6.438271
10    -3.079511
11     0.514376
12     5.939948
13    -7.672283
14    -4.817293
15     1.016507
16     7.496994
17     3.835590
18    -2.544202
19     5.866015
20     4.459769
21    -4.813318
22    -2.074645
23    -4.443277
24     3.888330
25     8.621597
26     2.941438
27   -14.954960
28    -4.591891
29     2.552398
30    -4.036982
31    -3.345244
32     6.916526
33    -5.428503
34   -10.062143
35     6.146851
36    -0.700177
37     8.793128
38     7.703242
39    -2.632472
40    10.472123
41     0.055520
42    -0.768592
43     1.977996
44     7.517355
45    14.550048
46    -8.491760
47    -3.245148
48    -2.350879
49     7.427594
50   -10.105471
51   -13.849010
52    -3.350515
53     3.102199
54    -9.432846
55    -0.199954
dtype: float64